# Batch Processing: Find Local Max and Separate Foci

**Daryna Yakymenko** <br> <br> 
31-07-2025

*Created for Biological Data Science Summer School, <br> Uzhhorod, Ukraine, July 19 - August 2, 2025*

In [2]:
from skimage.feature import peak_local_max
from skimage.io import imread, imshow
from skimage.filters import gaussian 
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.segmentation import watershed
import ipywidgets as widgets
from IPython.display import display
import tifffile
import numpy as np
import pandas as pd
import glob
import os
from datetime import datetime

In [13]:
today_date = datetime.now().strftime("%d_%m_%Y") # data

#output_folder_name = today_date + "_cells"

current_directory = os.getcwd()
#output_folder_path = os.path.join(current_directory, "output\\", output_folder_name) # pełna ścieżka do kalaloga
output_folder_path = os.path.join(current_directory, "data", "labels")
input_folder_path = os.path.join(current_directory, "data", "input")
masks_folder_path = os.path.join(current_directory, "data", "masks")
hist_folder_path = os.path.join(current_directory, "data", "histograms")


folders_to_check = [output_folder_path, input_folder_path,masks_folder_path, hist_folder_path]

for folder in folders_to_check:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Created folder: {folder}")
    else:
        print(f"Folder already exists: {folder}")

Folder already exists: C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\labels
Folder already exists: C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\input
Folder already exists: C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\masks
Folder already exists: C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\histograms


In [14]:
# Listy do przechowywania przetworzonych DataFrame'ów
tif_files = [] 

#Iteration by files in a folder
for file_name in os.listdir(input_folder_path): 
    if file_name.lower().endswith(('.tif', '.tiff')): 
        file_path = os.path.join(input_folder_path, file_name)
        tif_files.append(file_path)  #add full path to a file
        
print("Files found:")
for f in tif_files:
    print(f)

Files found:
C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\input\2015.01.12_ETP_1h-1h_SII_Pos001_S001_0_C3.tif
C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\input\2015.01.17_ETP_1h-1h_G_Pos001_S001_17_C1.tif
C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\input\2015.01.17_ETP_1h-1h_G_Pos001_S001_17_C2.tif


In [17]:
for file in tif_files: 
    image = imread(file)
    img_name = os.path.basename(file)
    base_name, ext = os.path.splitext(img_name)
    print(f"Image name: {img_name}, shape: {image.shape}")

    mask_path = os.path.join(masks_folder_path, img_name)
    mask_name = base_name + '.tiff'
    mask_path2 = os.path.join(masks_folder_path, mask_name)
    
    if os.path.exists(mask_path):
        binary_mask = imread(mask_path)
        print(f"Image name: {img_name}, shape: {image.shape}. Mask for the image found ({binary_mask.shape})")
    elif os.path.exists(mask_path2):
        binary_mask = imread(mask_path2)
        print(f"Image name: {img_name}, shape: {image.shape}. Mask found: {mask_path2} ({binary_mask.shape})")
    else:
        print(f"Missing mask for the image {img_name}!")
        break
        
    #Gaussian blur - image preprocessing 
    preprocessed = gaussian(image, sigma=1.5, preserve_range=True)
    print(f"min: {preprocessed.min()}, max: {preprocessed.max()}")
    
    #Create histograms of intensities
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    axs[0].hist(preprocessed.ravel(), bins=100)
    axs[0].set_title("Histogram (linear scale)")
    axs[0].set_xlabel("Intensity")
    axs[0].set_ylabel("Count")

    axs[1].hist(preprocessed.ravel(), bins=100)
    axs[1].set_title("Histogram (log scale)")
    axs[1].set_xlabel("Intensity")
    axs[1].set_yscale('log')
    axs[1].set_xlim(left=0)
    plt.tight_layout()
    #save as png
    save_path = os.path.join(hist_folder_path, img_name.replace('.tif', '_hist.png').replace('.tiff', '_hist.png'))
    plt.savefig(save_path)
    plt.close()
    print(f"Histogram saved to: {hist_folder_path}")

    coordinates = peak_local_max(preprocessed, threshold_abs=4, footprint=np.ones((4, 4, 4)))
    print(f"Nr of max found: {len(coordinates)}")

    #image = 3D array with intensities
    #binary_mask = 3D binary mask (1 - object, 0 - background)
    #coordinates = list of coordinates of local maxima (e.g., [(z1,y1,x1), (z2,y2,x2), ...])

    #create labels (label image) from seed points
    markers = np.zeros_like(binary_mask, dtype=np.int32)
    for i, (z, y, x) in enumerate(coordinates, 1):  #enumerate starting from 1
        markers[z, y, x] = i

    #set the mask where the growth is allowed
    mask = binary_mask.astype(bool)

    #watershed, with the usage of the original image 
    labels = watershed(-image, markers=markers, mask=mask)

    #to save labels - folder "./labels" should be created
    labels_path = os.path.join(output_folder_path, img_name.replace('.tif', '_labels.tif').replace('.tiff', '_labels.tif'))
    tifffile.imwrite(labels_path, labels.astype(np.uint16))

    print(f"Labels saved for the image {img_name}.")

print(f"All done.")

Image name: 2015.01.12_ETP_1h-1h_SII_Pos001_S001_0_C3.tif, shape: (99, 512, 512)
Image name: 2015.01.12_ETP_1h-1h_SII_Pos001_S001_0_C3.tif, shape: (99, 512, 512). Mask for the image found ((99, 512, 512))
min: 0.0, max: 94.02137915033313
Histogram saved to: C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\histograms
Nr of max found: 195
Labels saved for the image 2015.01.12_ETP_1h-1h_SII_Pos001_S001_0_C3.tif.
Image name: 2015.01.17_ETP_1h-1h_G_Pos001_S001_17_C1.tif, shape: (99, 512, 512)
Image name: 2015.01.17_ETP_1h-1h_G_Pos001_S001_17_C1.tif, shape: (99, 512, 512). Mask found: C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\masks\2015.01.17_ETP_1h-1h_G_Pos001_S001_17_C1.tiff ((99, 512, 512))
min: 0.0, max: 0.0
Histogram saved to: C:\Users\evgen\OneDrive\Desktop\BDS3_2025\data\histograms
Nr of max found: 0
Labels saved for the image 2015.01.17_ETP_1h-1h_G_Pos001_S001_17_C1.tif.
Image name: 2015.01.17_ETP_1h-1h_G_Pos001_S001_17_C2.tif, shape: (99, 512, 512)
Image name: 2015.01.17_ETP_1h-1